In [3]:
# Get credentials
from IPython.utils import io
with io.capture_output() as captured:
    %run ../Introduction.ipynb
    
from datetime import datetime, timedelta
from sentenai import Sentenai
sentenai = Sentenai(host=host, port=port)

# Streams

Streams in Sentenai are equivalent to a single-column in a database table, indexed by time range instead of a primary key.

A stream is characterized by having:
* A unique path
* A type
* Metadata
* Stats
* Data

## Unique Path

A stream's unique path is accessed via its stream representation:

In [2]:
print(sentenai['test-2']['Boston'])

test-2/Boston


This path is formatted such that it can be included in template strings when writing programs using TSPL, Sentenai's stream processing language.

## Types
A stream's "type", as determined by Sentenai, can be found by accessing the stream's type property:

In [5]:
sentenai['test-2']['Boston'].type

'event'

The type `'event'` indicates this stream is an event stream. Event streams are inner nodes on the tree, where as leaves will tend to (but not always!) have non-event types that mark the stream as a value stream.

In [9]:
sentenai['test-2']['Boston']['53']['Humidity'].type

'float'

## Metadata

Sentenai supports per-stream metadata. To access a stream's metadata, you can use the `.meta` sub-object, which is another dictionary-like type:

In [10]:
sentenai['test-2']['Boston'].meta

""


To add metadata to a stream, you can set a key with a value. Sentenai supports typed metadata, so you can insert metadata as numbers, strings, dates, booleans, and more.

In [12]:
sentenai['test-2']['Boston'].meta['cold'] = True

Metadata retrieval uses the same item-getter method:

In [14]:
sentenai['test-2']['Boston'].meta['cold']

True

And just like a dictionary, metadata can be iterated on:

In [5]:
for key, value in sentenai['test-2']['Boston'].meta.items():
    print(key, value, sep=":\t")

cold:	True


Deleting a metadata value can also be done by key:

In [9]:
del sentenai['test-2']['Boston'].meta['cold']
'cold' in sentenai['test-2']['Boston'].meta

False

## Stats

Sentenai supports calculating various statistics over each value stream. These statistics can be calculated over all values in the stream, or over a subset of data within the stream. Example:

In [9]:
sentenai['test-2']['Boston', '53', 'Humidity'].stats.mean

0.51612854

* Numeric types support `mean`, `min`, `max`, `sum`, `mean`, and `std`.
* All types support `count` and `top`.
* Boolean types support `any` and `all`

All types support time range slicing:

In [11]:
sentenai['test-2']['Boston', '53', 'Humidity'].stats[datetime(2020, 1, 1) : datetime(2020, 1, 2)].mean

0.51612854

## Data

The `.data` property of each stream support data retrieval over a range of time.

In [15]:
sentenai['test-2']['Boston', '53', 'Humidity'].data[::5]

[{'start': numpy.datetime64('2020-01-01T02:00:00'),
  'value': 0.9294579,
  'end': numpy.datetime64('2020-01-01T02:00:00')},
 {'start': numpy.datetime64('2020-01-01T02:00:00'),
  'value': 0.8240803,
  'end': numpy.datetime64('2020-01-01T02:01:00')},
 {'start': numpy.datetime64('2020-01-01T02:01:00'),
  'value': 0.63314134,
  'end': numpy.datetime64('2020-01-01T02:01:00')},
 {'start': numpy.datetime64('2020-01-01T02:01:00'),
  'value': 0.03078459,
  'end': numpy.datetime64('2020-01-01T02:02:00')},
 {'start': numpy.datetime64('2020-01-01T02:02:00'),
  'value': 0.30318394,
  'end': numpy.datetime64('2020-01-01T02:02:00')}]